## Uncovering Momentum, Section 1

# Dissecting Momentum Performance

This notebook presents the results of Section 1 'Dissecting Momentum Performance' from the paper [Uncovering Momentum](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3502301). Overall, the paper aims to explore and explain the source of the momentum premium following a systematic divide-and-conquer approach proceeding from the momentum factor into the stock level. Section 1 focuses on the top-level momentum behavior by subdividing the factor performance along two dimensions: bull/bear market states and winners/losers deciles. The results, as confirmed by the notebook, demonstrate the heterogeneous behavior of momentum across these dimensions and hence suggest that each dissected case can be associated with different processes and requires independent evaluation. 

The notebook is structured as follows: 
<ol>
    <li><a href='#section_1'> Loading Data</a></li>
    <li><a href='#section_2'> Formatting Data</a></li>
    <li><a href='#section_3'> Identifying Bull and Bear Market States</a></li>
    <li><a href='#section_4'> Overview of Heterogenous Momentum Behavior</a></li>
    <li><a href='#section_5'> Momentum Analysis across Deciles and Bull/Bear States</a></li>
</ol>

In [1]:
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import empyrical as ep

import statsmodels.api as sm
from statsmodels import regression

Defining time interval (updated from to cover data through December 2020)

In [2]:
start_date, end_date = '1927-01-01', '2020-12-31'

<a id='section_1'></a>
## 1. Loading Data

Momentum performance is evaluated using the [K.R. French Data Library](https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html), specifically, the 'Fama/French 3 Factors' and '10 Portfolios Formed on Momentum' datasets. As the corresponding datasets consist of multiple sections, the csv files need to be preliminary edited for extracting the value-weighted monthly data.

In [9]:
ff_factors_file = "F-F_Research_Data_Factors.csv"
mom_file = "10_Portfolios_Prior_12_2.csv"

In [10]:
date_parser = lambda x: datetime.strptime(str(x), '%Y%m')

factors_df = pd.read_csv(ff_factors_file, index_col=0, parse_dates=True, date_parser=date_parser)
mom_df = pd.read_csv(mom_file, index_col=0, parse_dates=True, date_parser=date_parser)

<a id='section_2'></a>
## 2. Formatting Data

Formatting data encompases selecting the time interval, converting percents into returns, and subtracting the risk-free rate (RF) from momentum deciles to compute excess returns.

In [11]:
factors_df = factors_df.loc[start_date: end_date]
mom_df = mom_df.loc[start_date: end_date]

In [12]:
factors_df = factors_df*0.01
mom_df = mom_df*0.01

In [13]:
ex_mom_df = mom_df.sub(factors_df['RF'], axis=0)

In [14]:
ex_mom_df.rename(columns={'Hi PRIOR': 'W10', 'Lo PRIOR': 'L10'},  inplace=True)
ex_mom_df['MKT'] = factors_df['Mkt-RF']
ex_mom_df['WML10'] = ex_mom_df['W10'] - ex_mom_df['L10']

<a id='section_3'></a>
## 3. Identifying Bull & Bear Market Intervals

For subsequent analysis of heterogenous momentum behavior, this section creates a bear market mask to identify and select momentum data seperately for bull and bear market intervals. The corresponding bull market state intervals are identified based on cumulative two year past returns with nonnegative values as computed by [Daniel and Moskowitz (2016)](https://www.sciencedirect.com/science/article/pii/S0304405X16301490).

In [15]:
def moving_cum(symbol, window=24):
    return (1. + symbol).rolling(window).agg(lambda x: x.prod()) - 1.

In [16]:
mkt_bear_mask = moving_cum(ex_mom_df["MKT"])  < 0

ex_mom_bull_df = ex_mom_df.loc[~mkt_bear_mask]
ex_mom_bear_df = ex_mom_df.loc[mkt_bear_mask]

<a id='section_4'></a>
## 4. Overview of Heterogenous Momentum  Behavior

This section replicates (and updates time interval up to Dec 2020) Table 1 from the paper Uncovering Momentum. The results demonstrate the heterogeneous behavior of momentum depending on the decile and market state components. The corresponding table compares the performance of excess market (MKT), conventional winners-minus-losers momentum strategy computed from the top/bottom 10% portfolios (WML10), and the top decile winners (W10) and losers (L10) components. Reported statistics encompass the annual mean, volatility, max drawdown, alpha and beta for both the full and bull-only market states. 

According to the table, past winners represent the solo driver producing annual mean with positive beta greater than one during the bull state. Losers in the bull state in contrast generate a small annual mean as a byproduct of the difference between a negative alpha and beta of the winners level. During the bear state, the beta parameters of past winners are substantially reduced and, concurrently, the past loser’s beta is increased. 

In [17]:
mom_cols = ['MKT', 'WML10', 'W10', 'L10']
t1_cols = ['annual_mean', 'annual volatility', 'max drawdown', 'annual alpha', 'beta']

### 4.1 Full interval

In [18]:
am = ex_mom_df[mom_cols].mean()*12*100
av = ex_mom_df[mom_cols].apply(lambda x: ep.annual_volatility(x, period='monthly')*100)
md = ex_mom_df[mom_cols].apply(lambda x: ep.max_drawdown(x))
alpha = ex_mom_df[mom_cols].apply(lambda x: ep.alpha(x, ex_mom_df['MKT'], risk_free=0.0, annualization=1)*12*100)
beta = ex_mom_df[mom_cols].apply(lambda x: ep.beta(x, ex_mom_df['MKT'], risk_free=0.0))

In [19]:
t1_df = pd.concat([am, av, md, alpha, beta], axis=1, keys=t1_cols)
t1_df

,annual_mean,annual volatility,max drawdown,annual alpha,beta
MKT,8.137447,18.588356,-0.846853,0.000000,1.000000
WML10,13.815000,27.276723,-0.956156,18.281893,-0.548931
W10,15.004043,22.397556,-0.782865,6.666583,1.024579
L10,1.189043,34.278321,-0.998117,-11.615310,1.573510


### 4.2 Bull market

In [20]:
am_bull = ex_mom_bull_df[mom_cols].mean()*12*100
av_bull = ex_mom_bull_df[mom_cols].apply(lambda x: ep.annual_volatility(x, period='monthly')*100)
md_bull = ex_mom_bull_df[mom_cols].apply(lambda x: ep.max_drawdown(x))
alpha_bull = ex_mom_bull_df[mom_cols].apply(lambda x: ep.alpha(x, ex_mom_bull_df['MKT'], risk_free=0.0, annualization=1)*12*100)
beta_bull = ex_mom_bull_df[mom_cols].apply(lambda x: ep.beta(x, ex_mom_bull_df['MKT'], risk_free=0.0))

In [21]:
t1_bull_df = pd.concat([am_bull, av_bull, md_bull, alpha_bull, beta_bull], axis=1, keys=t1_cols)
t1_bull_df

,annual_mean,annual volatility,max drawdown,annual alpha,beta
MKT,12.444583,14.526333,-0.352146,0.000000,1.000000
WML10,15.105556,20.274717,-0.540432,15.626563,-0.041866
W10,20.140000,21.091896,-0.492584,4.351574,1.268699
L10,5.034444,23.999751,-0.755048,-11.274989,1.310565


<a id='section_5'></a>
##  5. Momentum Analysis across Deciles and Bull/Bear States

This section proceeds with Table 2 from the paper for delving further into examining the asymmetric performance of momentum deciles during the bull and bear market states. Reported in the table are the alpha and beta perameters with corresponding t-statistics for each decile momentum portfolio seperately across the two market states. 

Focusing first solely on the bull market, alpha exhibits a positive linear shape and beta has a U-shape pattern. The results reinforce past research that going long the winners and short the losers generates positive alpha and simultaneously closely hedges the market beta. The latter parameter however is significant across all portfolios, while the former has a significant t-statistic only for the top and bottom 3 deciles. During the bear market, alpha is not significant across all decile portfolios with beta gradually declining from L10 to W10.

In [22]:
t2_mom_columns = list(set(ex_mom_bull_df.columns) - set(['MKT', 'WML10']))
t2_mom_columns.sort()

In [26]:
t2_cols = ['alpha', 't-stat', 'beta', 't-stat']

### 5.1 Bull market

In [29]:
ex_mkt = ex_mom_bull_df['MKT'].values
x1 = sm.add_constant(ex_mkt)

t2_results = []

for i, column in enumerate(t2_mom_columns):
    y = ex_mom_bull_df[column].values
    model = regression.linear_model.OLS(y, x1).fit()   
    t2_results.append([model.params[0]*12*100,model.tvalues[0], model.params[1],model.tvalues[1]])
    
pd.DataFrame(t2_results, index=t2_mom_columns, columns=t2_cols)

,alpha,t-stat,beta,t-stat
L10,-11.274989,-6.351435,1.310565,38.248495
PRIOR 2,-4.352680,-3.479595,1.069154,44.280497
PRIOR 3,-2.425341,-2.357820,0.962327,48.468609
PRIOR 4,-0.435408,-0.532085,0.924219,58.513981
PRIOR 5,0.454905,0.628103,0.909384,65.051514
PRIOR 6,0.197812,0.319209,0.946337,79.116755
PRIOR 7,1.019932,1.504550,0.965999,73.826523
PRIOR 8,2.114663,2.964698,0.993096,72.132453
PRIOR 9,2.416458,2.940097,1.068275,67.338832
W10,4.351574,3.492259,1.268699,52.749536


### 5.2 Bear market

In [30]:
ex_mkt = ex_mom_bear_df['MKT'].values
x1 = sm.add_constant(ex_mkt)

t2_results = []

for i, column in enumerate(t2_mom_columns):
    y = ex_mom_bear_df[column].values
    model = regression.linear_model.OLS(y, x1).fit()   
    t2_results.append([model.params[0]*12*100,model.tvalues[0], model.params[1],model.tvalues[1]])
    
pd.DataFrame(t2_results, index=t2_mom_columns, columns=t2_cols)

,alpha,t-stat,beta,t-stat
L10,-0.489800,-0.096719,1.830303,34.664427
PRIOR 2,2.867357,0.817141,1.598763,43.698768
PRIOR 3,2.487181,0.938845,1.377969,49.887969
PRIOR 4,3.252085,1.447384,1.259714,53.772893
PRIOR 5,0.718229,0.369873,1.152804,56.939714
PRIOR 6,3.121683,1.915490,1.110436,65.351309
PRIOR 7,1.515547,0.973306,0.969807,59.735873
PRIOR 8,1.388029,0.900842,0.874717,54.448644
PRIOR 9,1.716282,0.917510,0.861303,44.161935
W10,2.958254,1.062324,0.799310,27.530028
